In [442]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from webdriver_manager.chrome import ChromeDriverManager
chromeOptions = webdriver.ChromeOptions()
import pandas as pd
import numpy as np
import math

In [ ]:
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options = chromeOptions)
driver.get('https://www.dipres.gob.cl/597/w3-multipropertyvalues-15168-35324.html#proyecto_programa')
elems = driver.find_elements("xpath", "//a[@href]")
links = []
for elem in elems:
    links.append(elem.get_attribute("href"))

In [ ]:
links = pd.DataFrame(links)
xlss = links[links[0].str.contains('xls')]
xlss[1] = xlss[0].str.extract('([0-9][0-9][0-9][0-9][0-9][0-9])')
xlss[1] = xlss[1].astype('int')
xlss = xlss[(xlss[1]>=289355)&(xlss[1]<=289717)]
xlss

In [ ]:
import time

In [ ]:
i = 0
for url in xlss[0]:
    print(url)
    file1 = pd.read_excel(url)
    file1.to_excel('xls/' +str(i)+ '.xls')
    i+=1
    time.sleep(2)

In [ ]:
import os
import pandas as pd
path = os.getcwd() + '/xls/'
files = os.listdir(path)


In [ ]:
lista = []
for f in files:
    data = pd.read_excel('xls/' + f)
    forma = list(data.shape)
    forma.append(f)
    lista.append(forma)


In [494]:
df = pd.DataFrame(lista)
df[df[1]==13]

,0,1,2
91,49,13,9.xls
155,49,13,152.xls
220,25,13,7.xls
230,29,13,6.xls
339,35,13,136.xls


In [ ]:
dftotal = pd.DataFrame(columns=['subtitulo', 'item', 'asignacion', 'n_asignacion', 'glosa',
       'moneda_nacional_ms', 'moneda_internacional_musd', 'n_subtitulo',
       'n_item', 'partida', 'n_partida', 'capitulo', 'n_capitulo', 'programa',
       'n_programa'])
       
for row in df.iterrows():
    print(row[1][1], row[1][2])
    if row[1][1] == 17:
        dft = pd.read_excel('xls/'+ row[1][2])
        n_partida = dft['Unnamed: 1'][3]
        n_capitulo = dft['Unnamed: 1'][6]
        n_programa = dft['Unnamed: 1'][9]
        partida = dft['Unnamed: 15'][1]
        capitulo = dft['Unnamed: 15'][4]
        programa = dft['Unnamed: 15'][7]

        # Seleccionamos solo las filas con datos presupuestarios
        last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
        dft2 = dft.iloc[10:last]
        dft2 = dft2[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']]
        dft2.dropna(how='all', inplace=True)

        # eliminamos las filas de agregación
        agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
        dft2 = dft2.drop(agregaciones)
        dft2.reset_index(drop=True, inplace=True)

        #subtítulos
        dft2['n_subtitulo'] = '' #nombre subtitulo
        borrar = []
        temp = dft2[dft2['Unnamed: 1'].notna()]
        for i, subt in temp['Unnamed: 1'].iteritems():
                if type(subt) is str:
                        if i == len(dft2['Unnamed: 1'])-1:
                                if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                                else:
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                        else: 
                                if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                        borrar.append(i)
                                        j=i+1
                                        while type(dft2.at[j, 'Unnamed: 1']) is float:
                                                dft2.at[j, 'Unnamed: 1'] = subt
                                                dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 1'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 6']
        dft2 = dft2[dft2['n_subtitulo']!='']
        dft2.reset_index(drop=True, inplace=True)

        #items
        dft2['n_item'] = '' #nombre item
        borrar = []
        temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
        for i, subt in temp['Unnamed: 3'].iteritems():
                if (type(subt) is str):
                        if i == len(dft2['Unnamed: 3'])-1:
                                if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                                else:
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                        else:        
                                if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                        borrar.append(i)
                                        j=i+1
                                        while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                                dft2.at[j, 'Unnamed: 3'] = subt
                                                dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 6']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 3'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 6']

        dft2 = dft2[~dft2['Unnamed: 4'].isna()]
        dft2 = dft2.iloc[1:]

        dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 6':'n_asignacion',\
        'Unnamed: 7':'glosa', 'Unnamed: 8': 'moneda_nacional_ms', 'Unnamed: 9':'moneda_internacional_musd'}, inplace=True)
        dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
        dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
        dft2['partida'] = partida
        dft2['n_partida'] = n_partida
        dft2['capitulo'] = capitulo
        dft2['n_capitulo'] = n_capitulo
        dft2['programa'] = programa
        dft2['n_programa'] = n_programa

        dftotal = dftotal.append(dft2)
        
    if row[1][1] == 15:
        dft = pd.read_excel('xls/'+ row[1][2])
        n_partida = dft['Unnamed: 1'][3]
        n_capitulo = dft['Unnamed: 1'][6]
        n_programa = dft['Unnamed: 1'][9]
        partida = dft['Unnamed: 13'][1]
        capitulo = dft['Unnamed: 13'][4]
        programa = dft['Unnamed: 13'][7]

        # Seleccionamos solo las filas con datos presupuestarios
        if dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list() == []:
                last = len(dft)
        else:
                last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
        dft2 = dft.iloc[10:last]
        dft2 = dft2[['Unnamed: 1','Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']]
        dft2.dropna(how='all', inplace=True)

        # eliminamos las filas de agregación
        agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
        dft2 = dft2.drop(agregaciones)
        dft2.reset_index(drop=True, inplace=True)
        
        #subtítulos
        dft2['n_subtitulo'] = '' #nombre subtitulo
        borrar = []
        temp = dft2[dft2['Unnamed: 1'].notna()]
        for i, subt in temp['Unnamed: 1'].iteritems():
                if type(subt) is str:
                        if i == len(dft2['Unnamed: 1'])-1:
                                if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                        else: 
                                if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                        borrar.append(i)
                                        j=i+1
                                        while type(dft2.at[j, 'Unnamed: 1']) is float:
                                                dft2.at[j, 'Unnamed: 1'] = subt
                                                dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 1'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
        dft2 = dft2[dft2['n_subtitulo']!='']
        dft2.reset_index(drop=True, inplace=True)

        #items
        dft2['n_item'] = '' #nombre item
        borrar = []
        temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
        for i, subt in temp['Unnamed: 3'].iteritems():
                if (type(subt) is str):
                        if i == len(dft2['Unnamed: 3'])-1:
                                if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                        else:        
                                if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                        borrar.append(i)
                                        j=i+1
                                        while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                                dft2.at[j, 'Unnamed: 3'] = subt
                                                dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 3'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
        
        dft2 = dft2[~dft2['Unnamed: 4'].isna()]
        dft2 = dft2.iloc[1:]
        dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 5':'n_asignacion',\
        'Unnamed: 6':'glosa', 'Unnamed: 7': 'moneda_nacional_ms'}, inplace=True)
        dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
        dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
        dft2['partida'] = partida
        dft2['n_partida'] = n_partida
        dft2['capitulo'] = capitulo
        dft2['n_capitulo'] = n_capitulo
        dft2['programa'] = programa
        dft2['n_programa'] = n_programa

        dftotal = dftotal.append(dft2)

    if row[1][1] == 13:
        dft = pd.read_excel('xls/'+ row[1][2])
        n_partida = dft['Unnamed: 1'][3]
        n_capitulo = dft['Unnamed: 1'][6]
        n_programa = dft['Unnamed: 1'][9]
        partida = dft['Unnamed: 11'][1]
        capitulo = dft['Unnamed: 11'][4]
        programa = dft['Unnamed: 11'][7]

        # Seleccionamos solo las filas con datos presupuestarios
        if dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list() == []:
                last = len(dft)
        else:
                last = dft.index[dft['Unnamed: 1'] == 'GLOSAS:'].to_list()[0]
        dft2 = dft.iloc[10:last]
        dft2 = dft2[['Unnamed: 1','Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6']]
        dft2.dropna(how='all', inplace=True)
        dft2.rename(columns={'Unnamed: 2':'Unnamed: 3', 'Unnamed: 3':'Unnamed: 4', 'Unnamed: 4':'Unnamed: 5', 'Unnamed: 5':'Unnamed: 6', 'Unnamed: 6':'Unnamed: 7'}, inplace=True)

        # eliminamos las filas de agregación
        agregaciones = dft2.index[(dft2['Unnamed: 1'].isna())&(dft2['Unnamed: 3'].isna())&(dft2['Unnamed: 4'].isna())].to_list()
        dft2 = dft2.drop(agregaciones)
        dft2.reset_index(drop=True, inplace=True)

        #subtítulos
        dft2['n_subtitulo'] = '' #nombre subtitulo
        borrar = []
        temp = dft2[dft2['Unnamed: 1'].notna()]
        for i, subt in temp['Unnamed: 1'].iteritems():
                if type(subt) is str:
                        if i == len(dft2['Unnamed: 1'])-1:
                                if (type(dft2.at[i, 'Unnamed: 3']) is float) or (type(dft2.at[i, 'Unnamed: 3']) is np.float64):
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                        else: 
                                if type(dft2.at[i+1, 'Unnamed: 3']) is str:
                                        borrar.append(i)
                                        j=i+1
                                        while type(dft2.at[j, 'Unnamed: 1']) is float:
                                                dft2.at[j, 'Unnamed: 1'] = subt
                                                dft2.at[j, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 1'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 3']=0
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_subtitulo'] = dft2.at[i, 'Unnamed: 5']
        dft2 = dft2[dft2['n_subtitulo']!='']
        dft2.reset_index(drop=True, inplace=True)

        #items
        dft2['n_item'] = '' #nombre item
        borrar = []
        temp = dft2[(dft2['Unnamed: 3'].notna())&(dft2['Unnamed: 3']!=0)]
        for i, subt in temp['Unnamed: 3'].iteritems():
                if (type(subt) is str):
                        if i == len(dft2['Unnamed: 3'])-1:
                                if (type(dft2.at[i, 'Unnamed: 4']) is float) or (type(dft2.at[i, 'Unnamed: 4']) is np.float64):
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                else:
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                        else:        
                                if (not math.isnan(dft2.at[i+1, 'Unnamed: 4'])) and (dft2.at[i+1, 'Unnamed: 4']!=0):
                                        borrar.append(i)
                                        j=i+1
                                        while (type(dft2.at[j, 'Unnamed: 3']) is float) or (type(dft2.at[j, 'Unnamed: 3']) is np.float64):
                                                dft2.at[j, 'Unnamed: 3'] = subt
                                                dft2.at[j, 'n_item'] = dft2.at[i, 'Unnamed: 5']
                                                j+=1
                                                if j >= (len(dft2['Unnamed: 3'])):
                                                        break
                                else:
                                        dft2.at[i, 'Unnamed: 4']=0
                                        dft2.at[i, 'n_item'] = dft2.at[i, 'Unnamed: 5']
        
        dft2 = dft2[~dft2['Unnamed: 4'].isna()]
        dft2 = dft2.iloc[1:]
        dft2.rename(columns={'Unnamed: 1': 'subtitulo', 'Unnamed: 3':'item', 'Unnamed: 4':'asignacion', 'Unnamed: 5':'n_asignacion',\
        'Unnamed: 6':'glosa', 'Unnamed: 7': 'moneda_nacional_ms'}, inplace=True)
        dft2.loc[dft2['n_item']=='', 'n_item'] = '-'
        dft2.loc[dft2['asignacion']==0, 'n_asignacion'] = '-'
        dft2['partida'] = partida
        dft2['n_partida'] = n_partida
        dft2['capitulo'] = capitulo
        dft2['n_capitulo'] = n_capitulo
        dft2['programa'] = programa
        dft2['n_programa'] = n_programa

        dftotal = dftotal.append(dft2)




In [517]:
dftotal.to_excel('output/consolidado.xlsx')